### Dipendenze

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.ticker as ticker

In [ ]:
MYPATH = os.getcwd() + '/data/data_total/'
TOLLERANCE = 0.01
metric_to_compare = 'f1-score'
time_column_name = 'time'
timeUsed_column_name = 'timeUsed'
random_state_column = 'randomState'
datasets_to_process = ['XGB', 'VARIANZA']
TEMPO = False
MAGGIORE = True
DATASET = 'default' #'default', 'selfBACK', '?'

if DATASET == 'default':
  NOMI_FILE = {
    'baseline': 'modelXGBtotal_baseline',
    'modello_base': 'modelXGBtotal_base',
    'modello_varianza': 'modelXGBtotal_varianza',
  }
elif DATASET == 'selfBACK':
  NOMI_FILE = {
    'baseline': 'selfBACK_modelXGBtotal_baseline',
    'modello_base': 'selfBACK_modelXGBtotal_base',
    'modello_varianza': 'selfBACK_modelXGBtotal_varianza',
  }
elif DATASET == '?':
  NOMI_FILE = {
    'baseline': '?_modelXGBtotal_baseline',
    'modello_base': '?_modelXGBtotal_base',
    'modello_varianza': '?_modelXGBtotal_varianza',
  }
else:
    raise ValueError("DATASET non valido. Scegliere tra 'default', 'selfBACK' o '?'")

In [ ]:
temp = pd.read_csv(MYPATH + NOMI_FILE['modello_base'] + '.csv', header=0)
POSIZIONI = temp['position'].unique()
PESI = temp['weight'].unique()

In [ ]:
def format_seconds_to_minutes(val,pos):
    if val < 60:
        return f"{val:.2f} s"
    else:
        minutes = val / 60
        return f"{minutes:.2f} min"

In [ ]:
for position_to_plot in POSIZIONI:
    try:
        df_singolo_full = pd.read_csv(MYPATH + NOMI_FILE['baseline'] + '.csv', header=0)
        if df_singolo_full.empty:
            raise FileNotFoundError(f"Nessun dato SINGOLO (XGB peso=1) per {position_to_plot}")

        required_cols_singolo = [metric_to_compare, timeUsed_column_name, random_state_column]

        df_pos = df_singolo_full[
            (df_singolo_full['position'] == position_to_plot) &
            (df_singolo_full['randomState'].isin(df_singolo_full['randomState'].unique()))
        ]
        grouped = df_pos.groupby(['randomState', 'timeUsed'])[metric_to_compare].mean()
        max_medie = grouped.groupby('randomState').max()
        baseline_max_f1_dict = max_medie.to_dict()

        all_other_dfs = []
        datasets_files = {
            'XGB': MYPATH +  NOMI_FILE['modello_base'] + '.csv',
            'VARIANZA': MYPATH +  NOMI_FILE['modello_varianza'] + '.csv',
        }
        temp_datasets_to_process = list(datasets_to_process)
        for name, filepath in datasets_files.items():
            if name in temp_datasets_to_process:
                try:
                    df_loaded = pd.read_csv(filepath, header=0)
                    df_loaded['dataset'] = name
                    df_loaded = df_loaded[df_loaded['position'] == position_to_plot]

                    required_cols = ['weight', timeUsed_column_name, metric_to_compare, random_state_column]
                    if not all(col in df_loaded.columns for col in required_cols):
                        print(f"Attenzione: Colonne mancanti in {filepath} ({name}). Salto.")
                        datasets_to_process.remove(name)
                        continue

                    cols_to_keep = ['weight', timeUsed_column_name, metric_to_compare, 'dataset', random_state_column]
                    if time_column_name in df_loaded.columns:
                        cols_to_keep.append(time_column_name)
                    all_other_dfs.append(df_loaded[cols_to_keep])

                except FileNotFoundError:
                    print(f"File non trovato {filepath} ({name})")
                    datasets_to_process.remove(name)
                except Exception as e:
                    print(f"Errore {filepath} ({name}) {e}")
                    datasets_to_process.remove(name)

        df_all_others = pd.concat(all_other_dfs, ignore_index=True)
        df_all_others = df_all_others[df_all_others['weight'].isin(PESI)]

    except FileNotFoundError as e:
        print(f"Errore caricamento file {e}")
        continue
    except ValueError as e:
        print(f"Errore nei dati {e}")
        continue
    except Exception as e:
        print(f"Errore generico caricamento dati {e}")
        continue

    # --- Matching timeUsed per ogni run ---
    results_individual = []
    unique_random_states = df_all_others[random_state_column].unique()

    for weight_val in PESI:
        df_all_others_current_weight = df_all_others[df_all_others['weight'] == weight_val]

        for dataset_name in datasets_to_process:
            for r_state in unique_random_states:
                current_f1_s_max = baseline_max_f1_dict.get(r_state, np.nan)
                if not np.isfinite(current_f1_s_max):
                    results_individual.append({ 'weight': weight_val, 'dataset': dataset_name, 'random_state': r_state, 'matching_timeUsed': np.nan, 'corresponding_time': np.nan }); continue

                df_current_run = df_all_others_current_weight[ (df_all_others_current_weight['dataset'] == dataset_name) & (df_all_others_current_weight[random_state_column] == r_state) ]
                if df_current_run.empty:
                    results_individual.append({ 'weight': weight_val, 'dataset': dataset_name, 'random_state': r_state, 'matching_timeUsed': np.nan, 'corresponding_time': np.nan }); continue

                current_f1_map = df_current_run.groupby(timeUsed_column_name)[metric_to_compare].mean()
                current_time_map = df_current_run.groupby(timeUsed_column_name)[time_column_name].mean()

                timeUsed_values_to_check = sorted(current_f1_map.index.unique())
                if not timeUsed_values_to_check:
                    results_individual.append({ 'weight': weight_val, 'dataset': dataset_name, 'random_state': r_state, 'matching_timeUsed': np.nan, 'corresponding_time': np.nan }); continue


                matching_timeUsed = np.nan
                corresponding_time = np.nan

                # Ciclo sui valori di timeUsed per trovare il match
                for t_used in timeUsed_values_to_check:
                    f1_d = current_f1_map.get(t_used)
                    if f1_d is not None and np.isfinite(f1_d):
                       if (f1_d >= current_f1_s_max if MAGGIORE else abs(current_f1_s_max - f1_d) <= TOLLERANCE):
                            matching_timeUsed = t_used
                            if current_time_map is not None:
                                corresponding_time = current_time_map.get(t_used)
                            break

                if np.isnan(matching_timeUsed):
                    max_t_used_for_run = timeUsed_values_to_check[-1]
                    matching_timeUsed = max_t_used_for_run
                    if current_time_map is not None:
                        corresponding_time = current_time_map.get(max_t_used_for_run)

                results_individual.append({
                    'weight': weight_val,
                    'dataset': dataset_name,
                    'random_state': r_state,
                    'matching_timeUsed': matching_timeUsed,
                    'corresponding_time': corresponding_time
                })

    df_individual = pd.DataFrame(results_individual)
    df_average = df_individual.groupby(['weight', 'dataset'])[['matching_timeUsed', 'corresponding_time']].mean().reset_index()

    plot_data_y1_avg = df_average.dropna(subset=['matching_timeUsed'])
    plot_data_y1_ind = df_individual.dropna(subset=['matching_timeUsed'])

    if not plot_data_y1_avg.empty:
        fig, ax1 = plt.subplots(figsize=(24, 8))
        unique_datasets = df_average['dataset'].unique()
        color_palette = sns.color_palette(n_colors=len(unique_datasets))
        dataset_colors = dict(zip(unique_datasets, color_palette))

        sns.lineplot(
            data=plot_data_y1_ind, x='weight', y='matching_timeUsed', hue='dataset',
            palette=dataset_colors, units='random_state', estimator=None,
            linewidth=1, alpha=0.4, legend=False, ax=ax1
        )
        sns.lineplot(
            data=plot_data_y1_avg, x='weight', y='matching_timeUsed', hue='dataset',
            palette=dataset_colors, style='dataset', marker='o', markersize=9, dashes=False,
            linewidth=2, legend='full', ax=ax1
        )

        ax1.set_xlabel("Peso")
        ax1.set_xscale('log')
        ax1.set_xticks(PESI)
        ax1.set_xticklabels(PESI)
        y1_axis_label = f"timeUsed where F1 >= F1 Max baseline"
        ax1.set_ylabel(y1_axis_label, color='tab:blue')
        ax1.tick_params(axis='y', labelcolor='tab:blue')
        ax1.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, alpha=0.6)
        ax1.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5, alpha=0.6)
        ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_seconds_to_minutes))
        ymax = df_singolo_full[df_singolo_full['position'] == position_to_plot]['timeUsed'].max()
        ax1.set_ylim(-60, ymax+60)


        ax2 = None
        plot_data_y2_avg = df_average.dropna(subset=['corresponding_time'])
        plot_data_y2_ind = df_individual.dropna(subset=['corresponding_time'])

        if not plot_data_y2_avg.empty and TEMPO:
            ax2 = ax1.twinx()
            sns.lineplot(
                data=plot_data_y2_ind, x='weight', y='corresponding_time', hue='dataset',
                palette=dataset_colors, units='random_state', estimator=None,
                linewidth=1, alpha=0.4, legend=False, ax=ax2, linestyle='dotted'
            )
            sns.lineplot(
                data=plot_data_y2_avg, x='weight', y='corresponding_time', hue='dataset',
                palette=dataset_colors, style='dataset', marker='s', markersize=9, dashes=False,
                linewidth=2, legend=False, ax=ax2, linestyle='dotted'
            )

            ax2.set_ylabel(f'Tempo Addestramento Corrispondente ({time_column_name})', color='tab:red')
            ax2.tick_params(axis='y', labelcolor='tab:red')
            ax2.yaxis.set_major_formatter(ticker.FuncFormatter(format_seconds_to_minutes))

        tick_interval_seconds = 5 * 60
        yticks = np.arange(0, ymax + tick_interval_seconds, tick_interval_seconds)
        ax1.set_yticks(yticks)

        handles, labels = ax1.get_legend_handles_labels()
        num_datasets = len(unique_datasets)
        handles, labels = handles[:num_datasets], labels[:num_datasets]
        ax1.legend(handles, labels, title='Dataset (Media)', bbox_to_anchor=(1.18, 1), loc='upper left')

        plt.title(f'Posizione: {position_to_plot}')
        fig.tight_layout(rect=[0, 0, 0.82, 1])
        plt.show()
        plt.close()